In [28]:
import numpy as np
import pandas as pd

train=pd.read_csv('train.csv')
train.drop(train.columns[[0]], axis=1,inplace=True)
test=pd.read_csv('test.csv')
test.drop(test.columns[[0]], axis=1,inplace=True)

a=train.copy()
a['prom_times']=a['prom_1']+a['prom_4']+a['prom_6']+a['prom_10']
a=a.fillna({'prom_times':0})
a['prom_ok']=a.prom_times.map(lambda x: 1 if x>=1 else 0)
a['prom_type']=a['prom_1']*1000+a['prom_4']*100+a['prom_6']*10+a['prom_10']*1
a=a.fillna(value={'prom_type':0})
train=a.copy()


a=test.copy()
a['prom_times']=a['prom_1']+a['prom_4']+a['prom_6']+a['prom_10']
a=a.fillna({'prom_times':0})
a['prom_ok']=a.prom_times.map(lambda x: 1 if x>=1 else 0)
a['prom_type']=a['prom_1']*1000+a['prom_4']*100+a['prom_6']*10+a['prom_10']*1
a=a.fillna(value={'prom_type':0})
test=a.copy()

quantile=pd.read_csv('sku_quantile.csv')
quantile.columns = ['sku_id','quantile']

tt=pd.merge(train,quantile,on=['sku_id'],how='left')
ttest=pd.merge(test,quantile,on=['sku_id'],how='left')


In [32]:
def cacquantile(a,qi,i):
    qi=np.array(qi)[0]#！！！！
    aa={}
    for k in a:
        if k in aa.keys():
            aa[k]=aa[k]+1
        else:
            aa[k]=1
    
    lena=len(a)
    gradL=-qi*lena
    ans=0
    if lena==1:
        ans=a[0]
    if lena==2:
        ans=max((qi*a[0]+(1-qi)*a[1]),(qi*a[1]+(1-qi)*a[0]))
    if lena>=3:
        uu=list(set(a))
        uu=sorted(uu)
        index=0
        for k in uu:
            y1=gradL
            gradL+=aa[k]
            if gradL>0:
                if index==lena:#处理一下边界条件:最后才反转的
                    x1=uu[index-1]
                    x2=uu[index]
                else:
                    x1=(k+uu[index-1])/2
                    x2=(k+uu[index+1])/2
                y2=gradL
                ans=-y1*(x1-x2)/(y1-y2)+x1#与x轴交点
                
                break
            index+=1
    return ans 

def getset(idset,retset,n,i=0,keyset=[]):
    if len(keyset)==n:
        kk=list(keyset)#桥重点啊！！！！！！！，不然奇奇怪怪bug
        retset.append(kk)
        return 
    for k in range(i,len(idset)):
        key=idset[k]
        if key not in keyset:
            keyset.append(key)
            getset(idset,retset,n,k,keyset)
            keyset.remove(key)#与上面对应
        
def get_setid(idset,flag=0):
    retset=[] 
    chooseone=['first','second','third','sku_id']
    oneset=[]
    for i in chooseone :
        if i in idset:
            idset.remove(i)
            oneset.append(i)
    for i in range(1,len(idset)+1):
        qq=[]
        getset(idset,qq,i)
        retset.extend(qq)
    rretset=list(retset)
    for k in retset:
        for i in oneset:
            u=list(k)
            u.append(i)
            rretset.append(u) 
            rretset.append([i]) 
    #去重
    reset=[]
    for i in rretset:
        if i not in reset:
            reset.append(i)
            
    
    if flag!=0:
        #去掉过于复杂特征
        return reset
        types={'sku_id':1000,
             'dc_id':6,
             'first':3,
               'second':19,
               'third':96,
               'brand':378,
               'date':31,
               'prom_type':23
            }
        ans=[]
        for i in reset:
            times=1
            for key in i:
                times*=types[key]
            if times<=10000:
                ans.append(i)
        return ans
    else:
        return reset
def getonefeature(key,idset,va,f,data,mark):
    a=pd.DataFrame()
    name=mark
    for i in key:
        a.loc[:,i]=data[i]
        name+='_'+idset[i]
    name+='_'+va
    a.loc[:,name]=data[va]
    a=pd.pivot_table(a,index=key,aggfunc=f)
    a=a.reset_index()
    #print name
    return a
def getfeature(idset,values,data,target,mark=''):
    setid=get_setid(idset.keys())
    retdf=target.copy()
    GGG=['prom_1','prom_4','prom_6','prom_10','prom_ok','prom_times']
    for key in setid:
        for va in values.keys():
            if ('prom_type' not in key) or (('prom_type' in key) & (va not in GGG)):
                name=mark
                for i in key:
                    name+='_'+idset[i]
                name+='_'+va
                if name not in retdf:
                    a=getonefeature(key,idset,va,values[va],data,mark)
                    if len(setid)==1:
                        oninf=key[0]
                    else:
                        oninf=key
                    retdf = pd.merge(retdf, a, how='left',on=oninf)
    return retdf
        
def changetime(time):
    year,month,day=time.split('-')
    year=str(int(year)-1)
    time=year+'-'+month+'-'+day
    #print time
    return time
def cacquantile(data):
    a=np.array(data['quantity'])
    q=np.array(data['quantile'])
    
    aa={}
    for k in a:
        if k in aa.keys():
            aa[k]=aa[k]+1
        else:
            aa[k]=1
    
    lena=len(a)
    gradL=0
    for qi in q:
        gradL-=qi
    ans=0
    uu=list(set(a))
    uu=sorted(uu)
    if len(uu)==1:
        ans=uu[0]
    if len(uu)>=2:
        for index in range(len(uu)):
            y1=gradL
            gradL+=aa[uu[index]]
            if gradL>0:
                if index==len(uu)-1:#处理一下边界条件:最后才反转的
                    x1=uu[index-1]
                    x2=uu[index]
                else:
                    x1=(uu[index]+uu[index-1])/2
                    x2=(uu[index]+uu[index+1])/2
                y2=gradL
                ans=-y1*(x1-x2)/(y1-y2)+x1#与x轴交点
                
                break
    return ans 
def ccqi(keyset,data,mark):#计算qi分位点
    name=mark
    dddict={'sku_id':'sku',
     'dc_id':'dc',
     'first':'first',
       'second':'second',
       'third':'third',
       'brand':'brand',
        'prom_type':'promt'
    }
    a=pd.DataFrame()
    for i in keyset:
        name+='_'+dddict[i]
        a[i]=data[i]
    a[name]=data['quantity']+data['quantile']
    a=a.groupby(keyset)[name].unique().reset_index()
    def transform(x):
        a=pd.DataFrame(x,columns=['qi_qu'])
        a['quantity']=a['qi_qu'].map(lambda x:int(x))
        a['quantile']=a['qi_qu']-a['quantity']
        #print a
        return cacquantile(a)
    #print a
    a.loc[:,name]=a[name].map(transform)
    return a
                
def cacqiquan(idset,data,target,mark,quantile):
    setid=get_setid(idset.keys(),1)
    for keyset in setid:
        
        name=mark
        dddict={'sku_id':'sku',
         'dc_id':'dc',
         'first':'first',
           'second':'second',
           'third':'third',
           'brand':'brand',
            'prom_type':'promt'
        }
        for i in keyset:
            name+='_'+dddict[i]
            
        if name not in target:
            a=ccqi(keyset,data,mark)
            if len(a)==0:
                print a
            if len(a)>0:
                target=pd.merge(target, a, how='left',on=keyset)
        print name,':done'
    print "qi_dine!"
    return target
def do_it(full,begin,finish,flag=0,testhist=0):
    if flag==0:#验证集
        train=full[(full.date>=begin)&(full.date<=finish)]
        extend=full[(full.date>=changetime(begin))&(full.date<begin)]
        
        train=train.reset_index()
        train.drop(['index'],axis=1,inplace=True)
        extend=extend.reset_index()
        extend.drop(['index'],axis=1,inplace=True)
        
        if begin=='2017-01-01':
            begin='2016-12-01'
            finish='2016-12-31'
        if begin=='2017-12-01':
            begin='2017-10-01'
            finish='2017-10-31'
        #print begin,finish
        extend_last_month=full[(full.date>=begin)&(full.date<=finish)]
        extend_last_month=extend_last_month.reset_index()
        extend_last_month.drop(['index'],axis=1,inplace=True)
        
        #去除测试集没有的属性
        train.drop(['vend','or_price','discount'],axis=1,inplace=True)
        #清除 sum_quantity=0的数据
        dd=train

        a=pd.DataFrame()
        a['sku_id']=dd.sku_id
        a['y']=dd.quantity
        a['dc_id']=dd.dc_id
        g=a.groupby(['sku_id','dc_id'])['y'].sum().reset_index()
        del a['y']
        a=pd.merge(a,g,how='left',on=['sku_id','dc_id'])
        
        train.drop(a[a.y==0].index,inplace=True)
        train=train.reset_index()
        train.drop(['index'],axis=1,inplace=True)
        #构造关于分位点参数便于自定义损失函数
        z=train
        info=pd.DataFrame()
        info['sku_id']=z.sku_id
        info['quantity']=z.quantity
        info['dc_id']=z.dc_id

        q=info.groupby(['sku_id','dc_id'])['quantity'].sum().reset_index()
        q.rename(columns={'quantity':'sum_quantity'}, inplace=True) 
        info=pd.merge(info,q,how='left',on=['sku_id','dc_id'])

        info['date']=z.date
        info['quantile']=z['quantile']

        #print info
        info['k1']=info['quantile']/info.sum_quantity
        info['k2']=(1-info['quantile'])/info.sum_quantity

    else:
        
        extend=testhist[(testhist.date>=begin)&(testhist.date<=finish)]
        extend=extend.reset_index()
        extend.drop(['index'],axis=1,inplace=True)
        
        if begin=='2017-01-01':
            begin='2017-12-01'
            finish='2017-12-31'
        
        extend_last_month=testhist[(testhist.date>=begin)&(testhist.date<=finish)]
        extend_last_month=extend_last_month.reset_index()
        extend_last_month.drop(['index'],axis=1,inplace=True)
        
        train=full
    
    print 'train:',train.shape
    print 'extend:',extend.shape
    print 'extend_last_month:',extend_last_month.shape
    if flag==0:
        print 'info:',info.shape
    
    
    
    idset={'sku_id':'sku',
     'dc_id':'dc',
     'first':'first',
       'second':'second',
       'third':'third',
       'brand':'brand',
        'prom_type':'promt'
    }
    ans1=cacqiquan(idset,extend,train,'qi',quantile)
    ans=cacqiquan(idset,extend_last_month,ans1,'qi_lastmonth',quantile)
    
    
    idset={'sku_id':'sku',
     'dc_id':'dc',
     'first':'first',
       'second':'second',
       'third':'third',
       'brand':'brand',
       'prom_type':'promt'
    }
    #1 mean 2 sum 
    values={'quantity':np.mean,
            'vend':np.sum,
            'or_price':np.mean,
            'discount':np.mean,
            'prom_1':np.sum,
            'prom_4':np.sum,
            'prom_6':np.sum,
            'prom_10':np.sum,
            'prom_times':np.mean,
            'prom_ok':np.sum,
            'attr':np.mean,
           'attr_values':np.mean,
           'values_rate':np.mean}
    train_hist=getfeature(idset,values,extend,ans,'hist')


    idset={'sku_id':'sku',
         'dc_id':'dc',
         'first':'first',
           'second':'second',
           'third':'third',
           'brand':'brand',
           'date':'date',
           'prom_type':'promt'
        }
    #1 mean 2 sum 
    values={
            'prom_1':np.sum,
            'prom_4':np.sum,
            'prom_6':np.sum,
            'prom_10':np.sum,
            'prom_times':np.mean,
            'prom_ok':np.sum,
            'attr':np.mean,
           'attr_values':np.mean,
           'values_rate':np.mean}
    train=getfeature(idset,values,train,train_hist,'now')
    
    
    
    idset={'sku_id':'sku',
     'dc_id':'dc',
     'first':'first',
       'second':'second',
       'third':'third',
       'brand':'brand',
        'prom_type':'promt'
    }
    #1 mean 2 sum 
    values={'quantity':np.mean,
            'vend':np.sum,
            'or_price':np.mean,
            'discount':np.mean,
            'prom_1':np.sum,
            'prom_4':np.sum,
            'prom_6':np.sum,
            'prom_10':np.sum,
            'prom_times':np.mean,
            'prom_ok':np.sum,
            'attr':np.mean,
           'attr_values':np.mean,
           'values_rate':np.mean}
    ttrain=getfeature(idset,values,extend_last_month,train,'lastmonth')
    
    
    print 'train_extend:',ttrain.shape
    print '\n\n'
    if flag==0:
        return ttrain,info
    else:
        return ttrain
train201701,info201701=do_it(tt,'2017-01-01','2017-01-31')
train201712,info201712=do_it(tt,'2017-12-01','2017-12-31')


    
    

train: (103047, 19)
extend: (728627, 22)
extend_last_month: (104176, 22)
info: (103047, 8)
qi_brand :done
qi_promt :done
qi_dc :done
qi_brand_promt :done
qi_brand_dc :done
qi_promt_dc :done
qi_brand_promt_dc :done
qi_brand_first :done
qi_first :done
qi_brand_second :done
qi_second :done
qi_brand_third :done
qi_third :done
qi_brand_sku :done
qi_sku :done
qi_promt_first :done
qi_promt_second :done
qi_promt_third :done
qi_promt_sku :done
qi_dc_first :done
qi_dc_second :done
qi_dc_third :done
qi_dc_sku :done
qi_brand_promt_first :done
qi_brand_promt_second :done
qi_brand_promt_third :done
qi_brand_promt_sku :done
qi_brand_dc_first :done
qi_brand_dc_second :done
qi_brand_dc_third :done
qi_brand_dc_sku :done
qi_promt_dc_first :done
qi_promt_dc_second :done
qi_promt_dc_third :done
qi_promt_dc_sku :done
qi_brand_promt_dc_first :done
qi_brand_promt_dc_second :done
qi_brand_promt_dc_third :done
qi_brand_promt_dc_sku :done
qi_dine!
qi_lastmonth_brand :done
qi_lastmonth_promt :done
qi_lastmonth_dc

In [33]:
train201801=do_it(ttest,'2017-01-01','2017-12-31',1,tt)

train: (186000, 18)
extend: (1398858, 22)
extend_last_month: (182189, 22)
qi_brand :done
qi_promt :done
qi_dc :done
qi_brand_promt :done
qi_brand_dc :done
qi_promt_dc :done
qi_brand_promt_dc :done
qi_brand_first :done
qi_first :done
qi_brand_second :done
qi_second :done
qi_brand_third :done
qi_third :done
qi_brand_sku :done
qi_sku :done
qi_promt_first :done
qi_promt_second :done
qi_promt_third :done
qi_promt_sku :done
qi_dc_first :done
qi_dc_second :done
qi_dc_third :done
qi_dc_sku :done
qi_brand_promt_first :done
qi_brand_promt_second :done
qi_brand_promt_third :done
qi_brand_promt_sku :done
qi_brand_dc_first :done
qi_brand_dc_second :done
qi_brand_dc_third :done
qi_brand_dc_sku :done
qi_promt_dc_first :done
qi_promt_dc_second :done
qi_promt_dc_third :done
qi_promt_dc_sku :done
qi_brand_promt_dc_first :done
qi_brand_promt_dc_second :done
qi_brand_promt_dc_third :done
qi_brand_promt_dc_sku :done
qi_dine!
qi_lastmonth_brand :done
qi_lastmonth_promt :done
qi_lastmonth_dc :done
qi_lastmon

In [34]:
train1=train201701
train2=train201712
test=train201801
info1=info201701
info2=info201712

In [35]:
print train1.shape
print train2.shape
print test.shape
print info1.shape
print info2.shape
#for i in sorted(list(train1.columns)):#看有哪些属性
    #print i

(103047, 1342)
(179879, 1342)
(186000, 1341)
(103047, 8)
(179879, 8)


In [36]:
train1.to_csv('train201701.csv',index=False)
info1.to_csv('info201701.csv',index=False)
train2.to_csv('train201712.csv',index=False)
info2.to_csv('info201712.csv',index=False)
test.to_csv('test201801.csv',index=False)

In [1]:
import pandas as pd
import numpy as np
info1 = pd.read_csv('info201701.csv')
info2 = pd.read_csv('info201712.csv')
train1 = pd.read_csv('train201701.csv')
train2 = pd.read_csv('train201712.csv')
test = pd.read_csv('test201801.csv')
print info1.head()

   sku_id  quantity  dc_id  sum_quantity        date  quantile        k1  \
0     854       0.0      4          19.0  2017-01-09      0.99  0.052105   
1      53       0.0      1          99.0  2017-01-24      0.99  0.010000   
2     374       2.0      4         162.0  2017-01-14      0.97  0.005988   
3     407       2.0      1         163.0  2017-01-07      0.96  0.005890   
4     681      17.0      0          77.0  2017-01-09      0.95  0.012338   

         k2  
0  0.000526  
1  0.000101  
2  0.000185  
3  0.000245  
4  0.000649  


In [2]:
train1=train1.drop(['sku_id','dc_id','date','first','second','third','brand','quantile'],axis=1)
train2=train2.drop(['sku_id','dc_id','date','first','second','third','brand','quantile'],axis=1)
test=test.drop(['sku_id','dc_id','date','first','second','third','brand','quantile'],axis=1)

In [5]:
print train1.shape
print train2.shape
print test.shape
print info1.shape
print info2.shape

(103047, 1334)
(179879, 1334)
(186000, 1333)
(103047, 8)
(179879, 8)


In [ ]:
def get_evafunc(info1,info2):
    def evafunc(pred, dmat):
        if pred.shape[0]==info1.shape[0]:
            info=info1
        if pred.shape[0]==info2.shape[0]:
            info=info2
        
        op=info
        
        n=pred.shape[0]

        label=np.array(op['quantity'])
        i=np.array(op['sku_id'])
        j=np.array(op['dc_id'])
        qi=np.array(op['quantile'])
        k1=np.array(op['k1'])
        k2=np.array(op['k2'])

        L=np.zeros((1001,6))
        N=np.zeros((1001,6))
        for ind in range(n):
            if label[ind]>pred[ind]:
                L[i[ind]][j[ind]]+=k1[ind]*(label[ind]-pred[ind])
                N[i[ind]][j[ind]]+=1
            else:
                L[i[ind]][j[ind]]+=k2[ind]*(pred[ind]-label[ind])
                N[i[ind]][j[ind]]+=1
        ans=0
        for i in range(1001):
            for j in range(6):
                if N[i][j]!=0:
                    ans+=L[i][j]/N[i][j]
        myeval=ans
        return 'my_eval',myeval
    return evafunc
def get_loss(info1,info2):
    def lossfunc(pred, dmat): #自定义损失函数
        if pred.shape[0]==info1.shape[0]:
            info=info1
        if pred.shape[0]==info2.shape[0]:
            info=info2
        n=pred.shape[0]
        
        delta=pred-np.array(info['quantity'])
        k1=np.array(info.k1)
        k2=np.array(info.k2)
        import math
        qiudao1=np.zeros(n)
        qiudao2=np.zeros(n)
        for i in range(n):
            e_delta=math.exp(delta[i])
            qiudao1[i]=(-k1[i]-k2[i])/(1+e_delta)+k2[i]
            qiudao2[i]=(k1[i]+k2[i])/((1+e_delta)**2)*e_delta
        grad=qiudao1 #梯度
        hess=qiudao2
        return grad,hess
    return lossfunc

import xgboost as xgb
import numpy as np
import pandas as pd
#from xgboost.sklearn import XGBClassifier
#from sklearn import cross_validation,metrics
#from sklearn.grid_search import GridSearchCV

import time 
start_time = time.time()
params={
'booster':'gbtree',
#'objective': 'reg:linear', #线性回归的问题
#'eval_metric':'auc',
'gamma':0.15,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'min_child_weight':3,
'max_depth':5, # 构建树的深度，越大越容易过拟合
'lambda':10,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本#0.7
'colsample_bytree':0.7, # 生成树时进行的列采样#0.7
'colsample_bylevel':0.7,
#'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.1, # 如同学习率
'tree_method':'exact',
'seed':10,
#'nthread':12
#'nthread':,# cpu 线程数
#'eval_metric': 'auc'
}
train_X=train1.drop(['quantity'],axis=1)
train_Y=train1['quantity']

test_X=train2.drop(['quantity'],axis=1)
test_Y=train2['quantity']


train_xgb=xgb.DMatrix(train_X,label=train_Y)
test_xgb=xgb.DMatrix(test_X,label=test_Y)
watchlist=[(train_xgb,'train'),(test_xgb,'test')]
model=xgb.train(params,train_xgb,num_boost_round=1000,feval=get_evafunc(info1,info2),obj=get_loss(info1,info2),evals=watchlist)

cost_time = time.time()-start_time
print "xgboost success!",'\n',"cost time:",cost_time,"(s)"

[0]	train-rmse:202.487	test-rmse:130.511	train-my_eval:88.6763	test-my_eval:155.061
[1]	train-rmse:202.469	test-rmse:130.487	train-my_eval:83.6071	test-my_eval:146.635
[2]	train-rmse:202.453	test-rmse:130.467	train-my_eval:79.359	test-my_eval:139.49
[3]	train-rmse:202.437	test-rmse:130.447	train-my_eval:77.1793	test-my_eval:136.324
[4]	train-rmse:202.421	test-rmse:130.428	train-my_eval:75.9799	test-my_eval:135.027
[5]	train-rmse:202.406	test-rmse:130.409	train-my_eval:74.8949	test-my_eval:133.829
[6]	train-rmse:202.393	test-rmse:130.394	train-my_eval:74.085	test-my_eval:132.804
[7]	train-rmse:202.378	test-rmse:130.39	train-my_eval:73.2286	test-my_eval:132.848
[8]	train-rmse:202.366	test-rmse:130.375	train-my_eval:72.4117	test-my_eval:131.754
[9]	train-rmse:202.353	test-rmse:130.371	train-my_eval:71.6218	test-my_eval:131.879
[10]	train-rmse:202.342	test-rmse:130.359	train-my_eval:70.9847	test-my_eval:131.371
[11]	train-rmse:202.329	test-rmse:130.343	train-my_eval:70.4189	test-my_eval:13

[98]	train-rmse:201.836	test-rmse:129.748	train-my_eval:64.2251	test-my_eval:140.825
[99]	train-rmse:201.833	test-rmse:129.744	train-my_eval:64.2035	test-my_eval:140.801
[100]	train-rmse:201.831	test-rmse:129.742	train-my_eval:64.1563	test-my_eval:140.806
[101]	train-rmse:201.829	test-rmse:129.739	train-my_eval:64.1221	test-my_eval:140.706
[102]	train-rmse:201.826	test-rmse:129.736	train-my_eval:64.1138	test-my_eval:140.732
[103]	train-rmse:201.824	test-rmse:129.733	train-my_eval:64.0727	test-my_eval:140.963
[104]	train-rmse:201.822	test-rmse:129.73	train-my_eval:64.0133	test-my_eval:141.004
[105]	train-rmse:201.819	test-rmse:129.726	train-my_eval:63.9777	test-my_eval:141.077
[106]	train-rmse:201.816	test-rmse:129.723	train-my_eval:63.9446	test-my_eval:141.1
[107]	train-rmse:201.814	test-rmse:129.721	train-my_eval:63.9232	test-my_eval:141.056
[108]	train-rmse:201.812	test-rmse:129.718	train-my_eval:63.898	test-my_eval:141.052
[109]	train-rmse:201.81	test-rmse:129.715	train-my_eval:63.8

[194]	train-rmse:201.658	test-rmse:129.531	train-my_eval:61.852	test-my_eval:144.592
[195]	train-rmse:201.656	test-rmse:129.529	train-my_eval:61.8126	test-my_eval:144.548
[196]	train-rmse:201.655	test-rmse:129.527	train-my_eval:61.8251	test-my_eval:144.679
[197]	train-rmse:201.654	test-rmse:129.527	train-my_eval:61.7884	test-my_eval:144.602
[198]	train-rmse:201.652	test-rmse:129.525	train-my_eval:61.7936	test-my_eval:144.765
[199]	train-rmse:201.651	test-rmse:129.524	train-my_eval:61.7748	test-my_eval:144.775
[200]	train-rmse:201.65	test-rmse:129.522	train-my_eval:61.7518	test-my_eval:144.751
[201]	train-rmse:201.648	test-rmse:129.52	train-my_eval:61.7399	test-my_eval:144.763
[202]	train-rmse:201.647	test-rmse:129.518	train-my_eval:61.7214	test-my_eval:144.809
[203]	train-rmse:201.645	test-rmse:129.516	train-my_eval:61.7107	test-my_eval:144.818
[204]	train-rmse:201.644	test-rmse:129.515	train-my_eval:61.7005	test-my_eval:144.8
[205]	train-rmse:201.642	test-rmse:129.513	train-my_eval:61

[290]	train-rmse:201.547	test-rmse:129.399	train-my_eval:60.5612	test-my_eval:145.274
[291]	train-rmse:201.545	test-rmse:129.397	train-my_eval:60.5567	test-my_eval:145.352
[292]	train-rmse:201.545	test-rmse:129.396	train-my_eval:60.5446	test-my_eval:145.314
[293]	train-rmse:201.544	test-rmse:129.395	train-my_eval:60.5179	test-my_eval:145.503
[294]	train-rmse:201.542	test-rmse:129.394	train-my_eval:60.5171	test-my_eval:145.521
[295]	train-rmse:201.542	test-rmse:129.393	train-my_eval:60.4968	test-my_eval:145.558
[296]	train-rmse:201.54	test-rmse:129.391	train-my_eval:60.4857	test-my_eval:145.714
[297]	train-rmse:201.539	test-rmse:129.39	train-my_eval:60.4539	test-my_eval:145.676
[298]	train-rmse:201.538	test-rmse:129.388	train-my_eval:60.4548	test-my_eval:145.774
[299]	train-rmse:201.537	test-rmse:129.387	train-my_eval:60.4425	test-my_eval:145.761
[300]	train-rmse:201.536	test-rmse:129.386	train-my_eval:60.4193	test-my_eval:145.737
[301]	train-rmse:201.535	test-rmse:129.385	train-my_eval

[386]	train-rmse:201.457	test-rmse:129.295	train-my_eval:59.5917	test-my_eval:145.999
[387]	train-rmse:201.456	test-rmse:129.293	train-my_eval:59.5855	test-my_eval:146.002
[388]	train-rmse:201.455	test-rmse:129.292	train-my_eval:59.5881	test-my_eval:146.095
[389]	train-rmse:201.454	test-rmse:129.291	train-my_eval:59.5757	test-my_eval:146.063
[390]	train-rmse:201.453	test-rmse:129.29	train-my_eval:59.5614	test-my_eval:145.995
[391]	train-rmse:201.452	test-rmse:129.288	train-my_eval:59.5654	test-my_eval:146.065
[392]	train-rmse:201.451	test-rmse:129.288	train-my_eval:59.5508	test-my_eval:146.054
[393]	train-rmse:201.451	test-rmse:129.287	train-my_eval:59.5289	test-my_eval:145.992
[394]	train-rmse:201.45	test-rmse:129.286	train-my_eval:59.5171	test-my_eval:145.949
[395]	train-rmse:201.449	test-rmse:129.285	train-my_eval:59.5092	test-my_eval:145.983
[396]	train-rmse:201.448	test-rmse:129.283	train-my_eval:59.4954	test-my_eval:145.928
[397]	train-rmse:201.447	test-rmse:129.282	train-my_eval

...

[578]	train-rmse:201.325	test-rmse:129.144	train-my_eval:58.2859	test-my_eval:148.097
[579]	train-rmse:201.325	test-rmse:129.144	train-my_eval:58.2734	test-my_eval:148.082
[580]	train-rmse:201.325	test-rmse:129.143	train-my_eval:58.2563	test-my_eval:148.051
[581]	train-rmse:201.324	test-rmse:129.143	train-my_eval:58.2351	test-my_eval:148.064
[582]	train-rmse:201.323	test-rmse:129.142	train-my_eval:58.2198	test-my_eval:147.992
[583]	train-rmse:201.322	test-rmse:129.141	train-my_eval:58.2059	test-my_eval:147.992
[584]	train-rmse:201.322	test-rmse:129.14	train-my_eval:58.2033	test-my_eval:147.996
[585]	train-rmse:201.321	test-rmse:129.139	train-my_eval:58.2004	test-my_eval:147.998
[586]	train-rmse:201.32	test-rmse:129.139	train-my_eval:58.1821	test-my_eval:147.997
[587]	train-rmse:201.32	test-rmse:129.139	train-my_eval:58.176	test-my_eval:147.964
[588]	train-rmse:201.319	test-rmse:129.138	train-my_eval:58.1855	test-my_eval:148.053
[589]	train-rmse:201.319	test-rmse:129.137	train-my_eval:5

[674]	train-rmse:201.274	test-rmse:129.088	train-my_eval:57.7846	test-my_eval:148.483
[675]	train-rmse:201.274	test-rmse:129.087	train-my_eval:57.7749	test-my_eval:148.499
[676]	train-rmse:201.273	test-rmse:129.087	train-my_eval:57.7776	test-my_eval:148.563
[677]	train-rmse:201.273	test-rmse:129.086	train-my_eval:57.7761	test-my_eval:148.57
[678]	train-rmse:201.272	test-rmse:129.086	train-my_eval:57.7746	test-my_eval:148.536
[679]	train-rmse:201.272	test-rmse:129.085	train-my_eval:57.7677	test-my_eval:148.746
[680]	train-rmse:201.271	test-rmse:129.084	train-my_eval:57.7672	test-my_eval:148.813
[681]	train-rmse:201.271	test-rmse:129.084	train-my_eval:57.7588	test-my_eval:148.8
[682]	train-rmse:201.27	test-rmse:129.083	train-my_eval:57.7521	test-my_eval:148.794
[683]	train-rmse:201.27	test-rmse:129.083	train-my_eval:57.7512	test-my_eval:148.795
[684]	train-rmse:201.269	test-rmse:129.082	train-my_eval:57.7434	test-my_eval:148.807
[685]	train-rmse:201.269	test-rmse:129.081	train-my_eval:57

[770]	train-rmse:201.233	test-rmse:129.043	train-my_eval:57.4246	test-my_eval:149.454
[771]	train-rmse:201.232	test-rmse:129.042	train-my_eval:57.4239	test-my_eval:149.436
[772]	train-rmse:201.232	test-rmse:129.042	train-my_eval:57.428	test-my_eval:149.453
[773]	train-rmse:201.232	test-rmse:129.041	train-my_eval:57.4091	test-my_eval:149.412
[774]	train-rmse:201.231	test-rmse:129.041	train-my_eval:57.4023	test-my_eval:149.415
[775]	train-rmse:201.231	test-rmse:129.04	train-my_eval:57.4083	test-my_eval:149.478
[776]	train-rmse:201.23	test-rmse:129.04	train-my_eval:57.4045	test-my_eval:149.484
[777]	train-rmse:201.23	test-rmse:129.039	train-my_eval:57.4047	test-my_eval:149.521
[778]	train-rmse:201.229	test-rmse:129.039	train-my_eval:57.4068	test-my_eval:149.527
[779]	train-rmse:201.229	test-rmse:129.038	train-my_eval:57.4089	test-my_eval:149.545
[780]	train-rmse:201.229	test-rmse:129.038	train-my_eval:57.4026	test-my_eval:149.559
[781]	train-rmse:201.229	test-rmse:129.038	train-my_eval:57

[866]	train-rmse:201.195	test-rmse:129.001	train-my_eval:57.102	test-my_eval:150.281
[867]	train-rmse:201.194	test-rmse:129.001	train-my_eval:57.1017	test-my_eval:150.288
[868]	train-rmse:201.194	test-rmse:129	train-my_eval:57.1005	test-my_eval:150.279
[869]	train-rmse:201.194	test-rmse:129	train-my_eval:57.0868	test-my_eval:150.267
[870]	train-rmse:201.193	test-rmse:128.999	train-my_eval:57.0887	test-my_eval:150.271
[871]	train-rmse:201.192	test-rmse:128.998	train-my_eval:57.0898	test-my_eval:150.279
[872]	train-rmse:201.192	test-rmse:128.998	train-my_eval:57.0882	test-my_eval:150.278
[873]	train-rmse:201.192	test-rmse:128.998	train-my_eval:57.0861	test-my_eval:150.29
[874]	train-rmse:201.191	test-rmse:128.997	train-my_eval:57.08	test-my_eval:150.289
[875]	train-rmse:201.191	test-rmse:128.997	train-my_eval:57.0704	test-my_eval:150.284
[876]	train-rmse:201.191	test-rmse:128.997	train-my_eval:57.0683	test-my_eval:150.212
[877]	train-rmse:201.191	test-rmse:128.996	train-my_eval:57.0618	t

[962]	train-rmse:201.159	test-rmse:128.961	train-my_eval:56.8694	test-my_eval:150.743
[963]	train-rmse:201.159	test-rmse:128.961	train-my_eval:56.8602	test-my_eval:150.736
[964]	train-rmse:201.158	test-rmse:128.96	train-my_eval:56.8541	test-my_eval:150.728
[965]	train-rmse:201.158	test-rmse:128.96	train-my_eval:56.8514	test-my_eval:150.749
[966]	train-rmse:201.157	test-rmse:128.959	train-my_eval:56.8602	test-my_eval:150.796
[967]	train-rmse:201.157	test-rmse:128.959	train-my_eval:56.8572	test-my_eval:150.77
[968]	train-rmse:201.157	test-rmse:128.958	train-my_eval:56.8481	test-my_eval:150.755
[969]	train-rmse:201.157	test-rmse:128.958	train-my_eval:56.856	test-my_eval:150.773
[970]	train-rmse:201.156	test-rmse:128.958	train-my_eval:56.8466	test-my_eval:150.745
[971]	train-rmse:201.156	test-rmse:128.958	train-my_eval:56.8387	test-my_eval:150.764
[972]	train-rmse:201.156	test-rmse:128.958	train-my_eval:56.8255	test-my_eval:150.875
[973]	train-rmse:201.155	test-rmse:128.957	train-my_eval:5

In [7]:
qq=model.get_fscore()
a=sorted(qq,key=lambda x:qq[x],reverse=True)
print len(a)
for i in a:
    print i,'-'*(60-len(i)),qq[i]

834
lastmonth_dc_sku_quantity ----------------------------------- 362
hist_dc_sku_quantity ---------------------------------------- 239
lastmonth_brand_dc_sku_quantity ----------------------------- 213
qi_lastmonth_dc_sku ----------------------------------------- 203
hist_brand_sku_quantity ------------------------------------- 152
lastmonth_brand_sku_quantity -------------------------------- 145
qi_lastmonth_brand_sku -------------------------------------- 144
hist_brand_dc_sku_quantity ---------------------------------- 139
qi_brand_sku ------------------------------------------------ 115
hist_promt_dc_sku_quantity ---------------------------------- 108
qi_lastmonth_brand_dc_sku ----------------------------------- 106
hist_sku_quantity ------------------------------------------- 96
lastmonth_sku_quantity -------------------------------------- 71
qi_lastmonth_sku -------------------------------------------- 65
qi_sku ------------------------------------------------------ 57
hist_brand

now_brand_dc_third_prom_ok ---------------------------------- 5
hist_third_prom_1 ------------------------------------------- 5
hist_brand_second_prom_ok ----------------------------------- 5
now_date_brand_third_attr_values ---------------------------- 5
hist_brand_promt_third_vend --------------------------------- 5
hist_promt_dc_third_attr ------------------------------------ 5
lastmonth_third_discount ------------------------------------ 5
lastmonth_brand_prom_times ---------------------------------- 5
hist_brand_prom_4 ------------------------------------------- 5
lastmonth_dc_sku_prom_times --------------------------------- 5
lastmonth_promt_dc_third_vend ------------------------------- 5
hist_brand_second_or_price ---------------------------------- 5
hist_brand_dc_discount -------------------------------------- 5
lastmonth_third_prom_1 -------------------------------------- 5
lastmonth_brand_prom_1 -------------------------------------- 5
lastmonth_brand_promt_sku_discount -----

In [ ]:
test_X=test
#test_Y=train2['quantity']


test_xgb=xgb.DMatrix(test_X)
test_pred=model.predict(test_xgb)
print test_pred.shape

In [ ]:
print test_pred
intest=pd.read_csv('test.csv')
intest.drop(test.columns[[0]], axis=1,inplace=True)



In [ ]:
u=pd.DataFrame()
cc=intest
u['date']=cc.date
u['dc_id']=cc.dc_id
u['item_sku_id']=cc.sku_id
u['quantity']=pd.Series(test_pred)


In [ ]:
u['date']=u.date.map(lambda x:int(x.split('-')[2]))
u['quantity']=u.quantity.map(lambda x: float('%.2f' % float(x)))
print u


In [ ]:
u.to_csv('v3_qi_month_R1000_bigstep.csv',index=False)

In [36]:
qq=model.get_fscore()
a=sorted(qq,key=lambda x:qq[x],reverse=True)
print len(a)
for i in a:
    print i,'-'*(60-len(i)),qq[i]

450
qi_sku ------------------------------------------------------ 506
qi_dc_sku --------------------------------------------------- 476
hist_dc_sku_quantity ---------------------------------------- 328
hist_brand_sku_quantity ------------------------------------- 285
hist_brand_dc_sku_quantity ---------------------------------- 188
hist_sku_quantity ------------------------------------------- 156
qi_brand ---------------------------------------------------- 82
qi_brand_dc ------------------------------------------------- 51
hist_brand_sku_discount ------------------------------------- 50
now_date_sku_prom_1 ----------------------------------------- 47
hist_brand_sku_prom_1 --------------------------------------- 46
hist_brand_dc_third_quantity -------------------------------- 42
hist_brand_third_quantity ----------------------------------- 42
hist_brand_quantity ----------------------------------------- 41
now_date_values_rate ---------------------------------------- 39
now_date_first_

hist_dc_first_vend ------------------------------------------ 1
hist_dc_sku_values_rate ------------------------------------- 1
hist_dc_prom_6 ---------------------------------------------- 1
now_date_second_prom_4 -------------------------------------- 1
prom_6 ------------------------------------------------------ 1
now_brand_dc_first_prom_6 ----------------------------------- 1
now_third_attr ---------------------------------------------- 1
hist_dc_first_prom_times ------------------------------------ 1
now_dc_second_attr ------------------------------------------ 1
now_date_first_values_rate ---------------------------------- 1
now_brand_date_dc_sku_prom_10 ------------------------------- 1
hist_dc_first_prom_ok --------------------------------------- 1
now_brand_date_third_prom_times ----------------------------- 1
now_date_first_prom_ok -------------------------------------- 1
now_second_prom_4 ------------------------------------------- 1
now_second_prom_1 ----------------------

In [194]:
print '['
for i in a:
    if qq[i]>=70:
        print "'"+i+"'",','
print ']'

[
'hist_brand_sku_quantity' ,
'hist_dc_sku_quantity' ,
'hist_brand_dc_sku_quantity' ,
'hist_brand_sku_discount' ,
'hist_brand_third_quantity' ,
'hist_brand_sku_prom_1' ,
'hist_dc_third_or_price' ,
'hist_brand_sku_prom_ok' ,
'hist_brand_sku_or_price' ,
'hist_brand_third_discount' ,
'hist_brand_sku_vend' ,
'hist_brand_prom_6' ,
'hist_brand_sku_prom_10' ,
'hist_brand_dc_prom_4' ,
'now_date_values_rate' ,
'hist_brand_third_vend' ,
'hist_brand_second_discount' ,
'hist_brand_third_or_price' ,
'hist_dc_sku_or_price' ,
'now_brand_sku_prom_1' ,
'hist_brand_vend' ,
'hist_brand_sku_prom_times' ,
'now_date_attr' ,
'hist_brand_dc_third_quantity' ,
'now_brand_third_prom_1' ,
'now_brand_sku_prom_10' ,
'hist_dc_sku_prom_times' ,
'hist_brand_dc_third_discount' ,
'now_date_dc_prom_1' ,
'now_dc_third_prom_ok' ,
'now_date_first_prom_6' ,
'hist_dc_sku_prom_1' ,
'hist_brand_dc_third_or_price' ,
'hist_brand_first_prom_6' ,
]


In [195]:
import_feature=[
'hist_brand_sku_quantity' ,
'hist_dc_sku_quantity' ,
'hist_brand_dc_sku_quantity' ,
'hist_brand_sku_discount' ,
'hist_brand_third_quantity' ,
'hist_brand_sku_prom_1' ,
'hist_dc_third_or_price' ,
'hist_brand_sku_prom_ok' ,
'hist_brand_sku_or_price' ,
'hist_brand_third_discount' ,
'hist_brand_sku_vend' ,
'hist_brand_prom_6' ,
'hist_brand_sku_prom_10' ,
'hist_brand_dc_prom_4' ,
'now_date_values_rate' ,
'hist_brand_third_vend' ,
'hist_brand_second_discount' ,
'hist_brand_third_or_price' ,
'hist_dc_sku_or_price' ,
'now_brand_sku_prom_1' ,
'hist_brand_vend' ,
'hist_brand_sku_prom_times' ,
'now_date_attr' ,
'hist_brand_dc_third_quantity' ,
'now_brand_third_prom_1' ,
'now_brand_sku_prom_10' ,
'hist_dc_sku_prom_times' ,
'hist_brand_dc_third_discount' ,
'now_date_dc_prom_1' ,
'now_dc_third_prom_ok' ,
'now_date_first_prom_6' ,
'hist_dc_sku_prom_1' ,
'hist_brand_dc_third_or_price' ,
'hist_brand_first_prom_6' ,
]
print len(import_feature)

34


In [197]:
tt=pd.DataFrame()
for i in import_feature:
    tt[i]=train201701_hist_now[i]
tt['quantity']=train201701_hist_now['quantity']
uu=pd.DataFrame()
for i in import_feature:
    uu[i]=train201801_hist_now[i]
print tt.shape
print uu.shape

(103047, 35)
(186000, 34)


In [243]:
def get_evafunc(info1,info2):
    def evafunc(pred, dmat):
        if pred.shape[0]==info1.shape[0]:
            info=info1
        if pred.shape[0]==info2.shape[0]:
            info=info2
        
        op=info
        
        n=pred.shape[0]
        
        #start_time1 = time.time()
        
        label=np.array(op['quantity'])
        i=np.array(op['sku_id'])
        j=np.array(op['dc_id'])
        qi=np.array(op['quantile'])
        k1=np.array(op['k1'])
        k2=np.array(op['k2'])
        
        #cost_time1 = time.time()-start_time1
        #start_time2 = time.time()
        
        L=np.zeros((1001,6))
        N=np.zeros((1001,6))
        for ind in range(n):
            if label[ind]>pred[ind]:
                L[i[ind]][j[ind]]+=k1[ind]*(label[ind]-pred[ind])
                N[i[ind]][j[ind]]+=1
            else:
                L[i[ind]][j[ind]]+=k2[ind]*(pred[ind]-label[ind])
                N[i[ind]][j[ind]]+=1
        ans=0
        for i in range(1001):
            for j in range(6):
                if N[i][j]!=0:
                    ans+=L[i][j]/N[i][j]
        myeval=ans
        
        #cost_time2 = time.time()-start_time2
        #print "cost time1:",cost_time1,"(s)","cost time2:",cost_time2,"(s)",cost_time1/cost_time2
        
        
        return 'my_eval',myeval
    return evafunc
def get_loss(info1,info2):
    def lossfunc(pred, dmat): #自定义损失函数
        if pred.shape[0]==info1.shape[0]:
            info=info1
        if pred.shape[0]==info2.shape[0]:
            info=info2
        n=pred.shape[0]
        
        delta=pred-np.array(info['quantity'])
        k1=np.array(info.k1)
        k2=np.array(info.k2)
        import math
        e_delta=np.zeros(n)
        qiudao1=np.zeros(n)
        qiudao2=np.zeros(n)
        for i in range(n):
            e_delta[i]=math.exp(delta[i])
        for i in range(n):
            qiudao1[i]=(-k1[i]-k2[i])/(1+e_delta[i])+k2[i]
            qiudao2[i]=(k1[i]+k2[i])/((1+e_delta[i])**2)*e_delta[i]
        grad=qiudao1 #梯度
        hess=qiudao2
        return grad,hess
    return lossfunc

import xgboost as xgb
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV

import time 
start_time = time.time()
params={
'booster':'gbtree',
#'objective': 'reg:linear', #线性回归的问题
#'eval_metric':'auc',
'gamma':0.15,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'min_child_weight':3,
'max_depth':5, # 构建树的深度，越大越容易过拟合
'lambda':10,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本#0.7
'colsample_bytree':0.7, # 生成树时进行的列采样#0.7
'colsample_bylevel':0.7,
#'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.01, # 如同学习率
'tree_method':'exact',
'seed':10,
#'nthread':12
#'nthread':,# cpu 线程数
#'eval_metric': 'auc'
}
train_X=tt.drop(['quantity'],axis=1)
train_Y=tt['quantity']

test_X=uu

train_xgb=xgb.DMatrix(train_X,label=train_Y,feature_names=train_X.columns)
test_xgb=xgb.DMatrix(test_X,label=test_Y,feature_names=test_X.columns)
watchlist=[(train_xgb,'train')]
model=xgb.train(params,train_xgb,num_boost_round=1000,feval=get_evafunc(info201701,info201801),obj=get_loss(info201701,info201801),evals=watchlist)

cost_time = time.time()-start_time
print "xgboost success!",'\n',"cost time:",cost_time,"(s)"

[0]	train-rmse:202.502	train-my_eval:93.5586
[1]	train-rmse:202.501	train-my_eval:93.0332
[2]	train-rmse:202.499	train-my_eval:92.5126
[3]	train-rmse:202.498	train-my_eval:92.0095
[4]	train-rmse:202.496	train-my_eval:91.4962
[5]	train-rmse:202.495	train-my_eval:91.0002
[6]	train-rmse:202.494	train-my_eval:90.4942
[7]	train-rmse:202.493	train-my_eval:90.0048
[8]	train-rmse:202.491	train-my_eval:89.5039
[9]	train-rmse:202.49	train-my_eval:89.0056
[10]	train-rmse:202.489	train-my_eval:88.512
[11]	train-rmse:202.487	train-my_eval:88.0201
[12]	train-rmse:202.486	train-my_eval:87.5323
[13]	train-rmse:202.485	train-my_eval:87.048
[14]	train-rmse:202.483	train-my_eval:86.5637
[15]	train-rmse:202.482	train-my_eval:86.0821
[16]	train-rmse:202.481	train-my_eval:85.6045
[17]	train-rmse:202.479	train-my_eval:85.1292
[18]	train-rmse:202.478	train-my_eval:84.6517
[19]	train-rmse:202.477	train-my_eval:84.1804
[20]	train-rmse:202.475	train-my_eval:83.7064
[21]	train-rmse:202.474	train-my_eval:83.2418
[

KeyboardInterrupt: 

In [207]:

op=info201701
#aans=pd.DataFrame()
#aans['label']=op['quantity']
#aans['pred']=pd.Series(pred)
#aans['i']=op['sku_id']
#aans['j']=op['dc_id']
#aans['qi']=op['quantile']
#aans['k1']=op.k1
#aans['k2']=op.k2
train_X=train201701_hist_now.drop(['quantity'],axis=1)
train_Y=train201701_hist_now['quantity']

test_X=train201712_hist_now.drop(['quantity'],axis=1)
test_Y=train201712_hist_now['quantity']

train_xgb=xgb.DMatrix(train_X,label=train_Y,feature_names=train_X.columns)
test_xgb=xgb.DMatrix(test_X,label=test_Y,feature_names=test_X.columns)
pred=model.predict(train_xgb)



In [232]:
import math
print label
print math.exp(label)

[ 0.  0.  2. ...,  0.  1.  0.]


TypeError: only length-1 arrays can be converted to Python scalars

In [236]:

start_time = time.time()

cost_time = time.time()-start_time
print "xgboost success!",'\n',"cost time:",cost_time,"(s)"

[-0.00013811  0.00010094  0.00018072 ...,  0.00387772  0.00303945
  0.00630096]


In [237]:
op=info201701
ans=pd.DataFrame()
        
ans['pred']=pd.Series(pred)
ans['delta']=ans['pred']-info['quantity']
ans['k1']=info.k1
ans['k2']=info.k2
import math
ans['e_delta']=ans.delta.map(lambda x:math.exp(x))
ans['qiudao1']=ans.apply(lambda x:(-x.k1-x.k2)/(1+x.e_delta)+x.k2,axis=1)
ans['qiudao2']=ans.apply(lambda x:(x.k1+x.k2)/((1+x.e_delta)**2)*x.e_delta,axis=1)
grad=np.array(ans.qiudao1) #梯度
hess=np.array(ans.qiudao2)

print grad

[-0.00013811  0.00010094  0.00018072 ...,  0.00387772  0.00303945
  0.00630096]
